In [2]:
import pandas as pd
import polars as pl

data = pd.read_csv('replenishment_rule.csv',encoding  = "gb18030")
data = pl.from_pandas(data)

In [3]:
## 仓库缩写转化函数
def adj_warehouse(x):
    if x.__contains__('花桥'):
        return 'HQ'
    elif x.__contains__('东莞'):
        return 'DG'
    elif x.__contains__('燕郊'):
        return 'YJ'
    else: return 'ADJ'

In [4]:
check_one = data[(data['safe_stock']!= 0)&(data['is_used']== '启用')].groupby(['item_code','warehouse']).agg({'operator':'count'}).reset_index()
check_one[check_one['operator']!= 1]

ValueError: Cannot __getitem__ on DataFrame with item: 'shape: (136084,)
Series: 'safe_stock' [bool]
[
	true
	true
	false
	true
	true
	true
	true
	true
	true
	true
	true
	true
	...
	true
	true
	true
	true
	true
	true
	true
	true
	true
	true
	true
	true
	true
]' of type: '<class 'polars.internals.series.series.Series'>'.

In [22]:
data[(data['safe_stock']!= 0)&(data['is_used']== '启用')&(data['item_code']== 2009467)]

,operator,safe_stock,coverage,is_used,start_time,end_time,lifestage,model_code,family_code,family_name,sports_code,warehouse,item_code,brand_name,sports_name
10030,WM-葛亮,10.0,10,启用,2022-08-25 00:00:00,2022-11-25 00:00:00,1.0,8352586,10778,????,6,酷武东莞,2009467,DOMYOS,?
27209,WM-葛亮,10.0,10,启用,2022-08-25 00:00:00,2022-11-25 00:00:00,1.0,8352586,10778,????,6,酷武燕郊,2009467,DOMYOS,?
44388,WM-葛亮,10.0,10,启用,2022-08-25 00:00:00,2022-11-25 00:00:00,1.0,8352586,10778,????,6,酷武花桥,2009467,DOMYOS,?
135892,WM-龚明超,3.0,3,启用,2023-02-10 00:00:00,2025-03-31 00:00:00,1.0,8352586,10778,????,6,酷武花桥,2009467,DOMYOS,?
135893,WM-龚明超,3.0,3,启用,2023-02-10 00:00:00,2025-03-31 00:00:00,1.0,8352586,10778,????,6,酷武燕郊,2009467,DOMYOS,?
135894,WM-龚明超,3.0,3,启用,2023-02-10 00:00:00,2025-03-31 00:00:00,1.0,8352586,10778,????,6,酷武东莞,2009467,DOMYOS,?


### ***安全库存设置产品与非设置产品拆转差异***

In [5]:
product_info = pl.read_csv('product_info.csv')
product_info = product_info.with_column(pl.col("picture_date").apply(lambda x: x[0:7]))

In [10]:
file_dir

'C:/Users/Administrator/Desktop/product_data_2022_0.csv'

In [20]:
import os
raw_data = pl.read_csv(r'C:\Users\Administrator\Desktop\product_data_2021_2022\product_data_2022_0.csv')

file_name = os.listdir(r'C:\Users\Administrator\Desktop\product_data_2021_2022')
year = '2022'
file_name = [ i for i in file_name if '2022'in i]
ge = 0
ge2= 0
ge3 =0
ge4 = 0
for i in file_name:
    print(i)
    file_dir = 'C:/Users/Administrator/Desktop/product_data_2021_2022/'+i
    raw_data = pl.read_csv(file_dir,ignore_errors= True)
    ## 生成年月日字段
    time_dimen = ["year", "month", "day"]
    raw_data = raw_data.with_columns(
        pl.col('paytime').str.split("-").alias("time_list")
    ).with_column(
        pl.struct([
            pl.col("time_list").arr.get(i).alias(time_dimen[i]) for i in range(3)
        ]).alias("time_list"), 
    ).unnest('time_list')
    raw_data = raw_data.with_column(pl.col("paytime").apply(lambda x: x[0:7]).alias("match_month"))
    raw_data = raw_data.join(product_info,left_on = ["match_month","product_id"],right_on =["picture_date","item_code"],how = 'left')
    ## safestock 发货情况
    ## 默认仓销售情况
    default_sales_data_safestock = raw_data.filter(pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).groupby(
                        ['paytime']).agg([pl.col("nums").sum(), ]).sort(['paytime'], reverse=True)
    ## 默认仓分月仓库的转单情况
    transfer_sales_data_safestock = raw_data.filter(pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).with_columns(
                         pl.col("warehouse").apply(lambda x: adj_warehouse(x)).alias("adj_warehouse")
                         ).filter(pl.col('adj_warehouse')!=pl.col('default_warehouse')).groupby(
                        ['paytime']).agg([pl.col("nums").sum(), ])
    transfer_sales_data_safestock = transfer_sales_data_safestock.rename({"nums": "transfer_nums"})

    ## 连接两表计算当天转单率
    sales_data_safestock_month = default_sales_data_safestock.join(transfer_sales_data_safestock, on=['paytime'], how="left")
    sales_data_safestock_month = sales_data_safestock_month.with_columns(
            [pl.col("transfer_nums").fill_null(pl.lit(0),).alias("transfer_nums"),]
        ).with_columns(
            [ (pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate"),]
        )
    if ge == 0:
         sales_data_safestock = sales_data_safestock_month.cleared()
    ge+=1
    sales_data_safestock = pl.concat([sales_data_safestock, sales_data_safestock_month])
    
    ## safestock set 0 发货情况
    ## 默认仓销售情况
    default_sales_data_safestock_zero = raw_data.filter(~pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).groupby(
                        ['paytime']).agg([pl.col("nums").sum(), ]).sort(['paytime'], reverse=True)
    ## 默认仓分月仓库的转单情况
    transfer_sales_data_safestock_zero = raw_data.filter(~pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).with_columns(
                         pl.col("warehouse").apply(lambda x: adj_warehouse(x)).alias("adj_warehouse")
                         ).filter(pl.col('adj_warehouse')!=pl.col('default_warehouse')).groupby(
                        ['paytime']).agg([pl.col("nums").sum(), ])
    transfer_sales_data_safestock_zero = transfer_sales_data_safestock_zero.rename({"nums": "transfer_nums"})

    ## 连接两表计算当天转单率
    sales_data_safestock_zero_month = default_sales_data_safestock_zero.join(transfer_sales_data_safestock_zero, on=['paytime'], how="left")
    sales_data_safestock_zero_month = sales_data_safestock_zero_month.with_columns(
            [pl.col("transfer_nums").fill_null(pl.lit(0),).alias("transfer_nums"),]
        ).with_columns(
            [ (pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate"),]
        )
    if ge2 == 0:
         sales_data_safestock_zero = sales_data_safestock_zero_month.cleared()
    ge2+=1
    sales_data_safestock_zero = pl.concat([sales_data_safestock_zero, sales_data_safestock_zero_month])
    
    
    
    ## safestock 详细内部不同产品信息下的差异
    ## 默认仓销售情况
    default_sales_data_safestock_info = raw_data.filter(pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).groupby(
                        ['match_month','item_lifestage','seasons']).agg([pl.col("nums").sum(), ]).sort(['match_month'], reverse=True)
    ## 默认仓分月仓库的转单情况
    transfer_sales_data_safestock_info = raw_data.filter(pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).with_columns(
                         pl.col("warehouse").apply(lambda x: adj_warehouse(x)).alias("adj_warehouse")
                         ).filter(pl.col('adj_warehouse')!=pl.col('default_warehouse')).groupby(
                        ['match_month','item_lifestage','seasons']).agg([pl.col("nums").sum(), ])
    transfer_sales_data_safestock_info = transfer_sales_data_safestock_info.rename({"nums": "transfer_nums"})

    ## 连接两表计算当天转单率
    sales_data_safestock_month_info = default_sales_data_safestock_info.join(transfer_sales_data_safestock_info, on=['match_month','item_lifestage','seasons'], how="left")
    sales_data_safestock_month_info = sales_data_safestock_month_info.with_columns(
            [pl.col("transfer_nums").fill_null(pl.lit(0),).alias("transfer_nums"),]
        ).with_columns(
            [ (pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate"),]
        )
    if ge3 == 0:
         sales_data_safestock_info = sales_data_safestock_month_info.cleared()
    ge3+=1
    sales_data_safestock_info = pl.concat([sales_data_safestock_info, sales_data_safestock_month_info])
    
    ## safestock set 0 发货情况
    ## 默认仓销售情况
    default_sales_data_safestock_zero_info = raw_data.filter(~pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).groupby(
                        ['match_month','item_lifestage','seasons']).agg([pl.col("nums").sum(), ]).sort(['match_month'], reverse=True)
    ## 默认仓分月仓库的转单情况
    transfer_sales_data_safestock_zero_info = raw_data.filter(~pl.col("product_id").is_in(list(data.filter((pl.col("safe_stock")!=0)&(pl.col("is_used")=="启用")).select(pl.col("item_code"))))).with_columns(
                         pl.col("warehouse").apply(lambda x: adj_warehouse(x)).alias("adj_warehouse")
                         ).filter(pl.col('adj_warehouse')!=pl.col('default_warehouse')).groupby(
                        ['match_month','item_lifestage','seasons']).agg([pl.col("nums").sum(), ])
    transfer_sales_data_safestock_zero_info = transfer_sales_data_safestock_zero_info.rename({"nums": "transfer_nums"})

    ## 连接两表计算当天转单率
    sales_data_safestock_zero_month_info = default_sales_data_safestock_zero_info.join(transfer_sales_data_safestock_zero_info, on=['match_month','item_lifestage','seasons'], how="left")
    sales_data_safestock_zero_month_info = sales_data_safestock_zero_month_info.with_columns(
            [pl.col("transfer_nums").fill_null(pl.lit(0),).alias("transfer_nums"),]
        ).with_columns(
            [ (pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate"),]
        )
    if ge4 == 0:
         sales_data_safestock_zero_info = sales_data_safestock_zero_month_info.cleared()
    ge4+=1
    sales_data_safestock_zero_info = pl.concat([sales_data_safestock_zero_info, sales_data_safestock_zero_month_info])    
    

product_data_2022_0.csv
product_data_2022_1.csv
product_data_2022_10.csv
product_data_2022_11.csv
product_data_2022_2.csv
product_data_2022_3.csv
product_data_2022_4.csv
product_data_2022_5.csv
product_data_2022_6.csv
product_data_2022_7.csv
product_data_2022_8.csv
product_data_2022_9.csv


In [41]:
summ_safestock = sales_data_safestock.select([pl.col("nums").sum(),pl.col("transfer_nums").sum()]).with_columns([(pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate"),pl.lit("safetystock_used").alias("safetystock_setting")]).with_columns(
                         pl.col("transfer_rate").apply(lambda x: format(x, '.2%'))).select([pl.col('safetystock_setting'), pl.col('nums'), pl.col('transfer_nums'), pl.col('transfer_rate')])
summ_safestock_zero = sales_data_safestock_zero.select([pl.col("nums").sum(),pl.col("transfer_nums").sum()]).with_columns([(pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate"),pl.lit("safetystock_close").alias("safetystock_setting")]).with_columns(
                         pl.col("transfer_rate").apply(lambda x: format(x, '.2%'))).select([pl.col('safetystock_setting'), pl.col('nums'), pl.col('transfer_nums'), pl.col('transfer_rate')])

summ_safestock_1 = pl.concat([summ_safestock, summ_safestock_zero])   

print(summ_safestock)

print(summ_safestock_zero)

print("从目前不考虑是否设置安全库存本身会产生的选择偏差，设置安全库存的产品相对未设置安全库存的产品拆转单率高于7.10%")


shape: (1, 4)
┌─────────────────────┬──────────┬───────────────┬───────────────┐
│ safetystock_setting ┆ nums     ┆ transfer_nums ┆ transfer_rate │
│ ---                 ┆ ---      ┆ ---           ┆ ---           │
│ str                 ┆ i64      ┆ i64           ┆ str           │
╞═════════════════════╪══════════╪═══════════════╪═══════════════╡
│ safetystock_used    ┆ 16468589 ┆ 4722356       ┆ 28.67%        │
└─────────────────────┴──────────┴───────────────┴───────────────┘
shape: (1, 4)
┌─────────────────────┬─────────┬───────────────┬───────────────┐
│ safetystock_setting ┆ nums    ┆ transfer_nums ┆ transfer_rate │
│ ---                 ┆ ---     ┆ ---           ┆ ---           │
│ str                 ┆ i64     ┆ i64           ┆ str           │
╞═════════════════════╪═════════╪═══════════════╪═══════════════╡
│ safetystock_close   ┆ 5835134 ┆ 1258854       ┆ 21.57%        │
└─────────────────────┴─────────┴───────────────┴───────────────┘
从目前不考虑是否设置安全库存本身会产生的选择偏差，设置安全库存的产品相对未设置安全

In [26]:
summ_safestock_stage = sales_data_safestock_info.filter(pl.col("match_month")<'2022-09').groupby(["item_lifestage"]).agg([pl.col("nums").sum(),pl.col("transfer_nums").sum()]).with_column((pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate")).with_columns(
                         pl.col("transfer_rate").apply(lambda x: format(x, '.2%')))
summ_safestock_zero_stage = sales_data_safestock_zero_info.filter(pl.col("match_month")<'2022-09').groupby(["item_lifestage"]).agg([pl.col("nums").sum(),pl.col("transfer_nums").sum()]).with_column((pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate")).with_columns(
                         pl.col("transfer_rate").apply(lambda x: format(x, '.2%')))
print(summ_safestock_stage)
print(summ_safestock_zero_stage)

shape: (7, 4)
┌──────────────────┬──────────┬───────────────┬───────────────┐
│ item_lifestage   ┆ nums     ┆ transfer_nums ┆ transfer_rate │
│ ---              ┆ ---      ┆ ---           ┆ ---           │
│ str              ┆ i64      ┆ i64           ┆ str           │
╞══════════════════╪══════════╪═══════════════╪═══════════════╡
│ DELETING         ┆ 169315   ┆ 64636         ┆ 38.17%        │
│ NEW              ┆ 34833    ┆ 13726         ┆ 39.41%        │
│ NO_STOCK_CAC_CAR ┆ 30184    ┆ 14252         ┆ 47.22%        │
│ NO_STOCK_CAC     ┆ 74595    ┆ 31863         ┆ 42.71%        │
│ null             ┆ 905      ┆ 365           ┆ 40.33%        │
│ DELETED          ┆ 105      ┆ 65            ┆ 61.90%        │
│ ACTIVE           ┆ 10894186 ┆ 3236337       ┆ 29.71%        │
└──────────────────┴──────────┴───────────────┴───────────────┘
shape: (7, 4)
┌──────────────────┬─────────┬───────────────┬───────────────┐
│ item_lifestage   ┆ nums    ┆ transfer_nums ┆ transfer_rate │
│ ---         

In [32]:
summ_safestock_season = sales_data_safestock_info.filter(pl.col("match_month")<'2022-09').groupby(["seasons"]).agg([pl.col("nums").sum(),pl.col("transfer_nums").sum()]).with_column((pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate")).with_columns(
                         pl.col("transfer_rate").apply(lambda x: format(x, '.2%'))).sort(['transfer_nums','transfer_rate'], reverse=True)
summ_safestock_zero_season = sales_data_safestock_zero_info.filter(pl.col("match_month")<'2022-09').groupby(["seasons"]).agg([pl.col("nums").sum(),pl.col("transfer_nums").sum()]).with_column((pl.col("transfer_nums")/pl.col("nums")).alias("transfer_rate")).with_columns(
                         pl.col("transfer_rate").apply(lambda x: format(x, '.2%'))).sort(['transfer_nums','transfer_rate'], reverse=True)
print(summ_safestock_season)
print(summ_safestock_zero_season)

shape: (172, 4)
┌─────────────────────────────────────┬─────────┬───────────────┬───────────────┐
│ seasons                             ┆ nums    ┆ transfer_nums ┆ transfer_rate │
│ ---                                 ┆ ---     ┆ ---           ┆ ---           │
│ str                                 ┆ i64     ┆ i64           ┆ str           │
╞═════════════════════════════════════╪═════════╪═══════════════╪═══════════════╡
│ 0                                   ┆ 8164260 ┆ 2486469       ┆ 30.46%        │
│ Code E                              ┆ 306113  ┆ 86604         ┆ 28.29%        │
│ SS19                                ┆ 229887  ┆ 81431         ┆ 35.42%        │
│ SS19,AW19                           ┆ 196911  ┆ 51065         ┆ 25.93%        │
│ ...                                 ┆ ...     ┆ ...           ┆ ...           │
│ AW15,SS16,SS16 (HK),AW16,AW16 (H... ┆ 6       ┆ 1             ┆ 16.67%        │
│ SS15,AW15,SS16,SS16 (HK),AW18,SS... ┆ 1       ┆ 1             ┆ 100.00%       │


### ***最小最大库存过往7天平均逻辑是否可以满足实际情况***

1.CAR有货的情况下

2.CAR没货的情况下


In [ ]:
主要销售产品的